In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from freezegun import freeze_time

from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.supabase_client import upsert_training_week_with_coaching, get_training_week_with_coaching
from src.training_week import generate_training_week
from src.email_manager import send_email, training_week_to_html

athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)

athlete_id='98390356' token still valid until 2024-08-12 00:18:57+00:00


# Sunday Night Training Week Message

In [4]:
@freeze_time("2024-08-04 20:00:00")
def mock_get_activities_df(strava_client):
    return get_activities_df(strava_client)

activities_df = mock_get_activities_df(strava_client)

day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
training_week = generate_training_week(
    client_preferences=client_preferences,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries
)

# upsert_training_week_with_coaching(
#     athlete_id=athlete_id, training_week_with_coaching=training_week
# )
# send_email(
#     subject="Training Schedule Just Dropped 🏃",
#     html_content=training_week_to_html(training_week),
# )

{'message_id': '<202408101610.79765560078@smtp-relay.mailin.fr>',
 'message_ids': None}

# Mid-Week Update

In [58]:
from src.types.training_week import TrainingWeek
from src.types.training_week_with_coaching import TrainingWeekWithCoaching
import polars as pl

def calc_weekly_total_mileage(training_week: TrainingWeek) -> float:
    """
    Calculates the total mileage for a given week

    :param training_week: instance of TrainingWeek
    :return: The total mileage
    """ 
    return sum(
        [
            getattr(training_week, attrib).distance 
            for attrib in training_week.__dict__
        ]
    )
    

def get_mid_week_delta(
    training_week_with_coaching: TrainingWeekWithCoaching,
    activities: pl.DataFrame
) -> float:
    """
    Assumes that activities is truncated to the current week, calculating the 
    difference between target mileage and actual mileage thus far this week

    :param training_week_with_coaching: instance TrainingWeekWithCoaching
    :param activities: DataFrame of activities thus far this week
    :return: The mileage delta as a float
    """
    actual_mileage = activities['distance_in_miles'].sum()
    weekly_target = calc_weekly_total_mileage(
        training_week_with_coaching.training_week
    )
    mileage_delta = weekly_target - actual_mileage

    return mileage_delta

In [53]:
@freeze_time("2024-08-08 20:00:00")
def mock_get_activities_df(strava_client, num_weeks=8):
    return get_activities_df(strava_client, num_weeks)

training_week_with_coaching = get_training_week_with_coaching(athlete_id)
activities = mock_get_activities_df(strava_client, num_weeks=1)

mid_week_detla = get_mid_week_delta(
    training_week_with_coaching=training_week_with_coaching,
    activities=activities
)